## Load dataset

In [1]:
!wget "https://raw.githubusercontent.com/usydnlp/Glocal_K/main/data/MovieLens_100K/movielens_100k_u1.base"
!wget "https://raw.githubusercontent.com/usydnlp/Glocal_K/main/data/MovieLens_100K/movielens_100k_u1.test"

--2023-03-20 13:57:05--  https://raw.githubusercontent.com/usydnlp/Glocal_K/main/data/MovieLens_100K/movielens_100k_u1.base
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1586544 (1.5M) [text/plain]
Saving to: ‘movielens_100k_u1.base’

movielens_100k_u1.b 100%[===================>]   1.51M  --.-KB/s    in 0.06s   

2023-03-20 13:57:06 (25.7 MB/s) - ‘movielens_100k_u1.base’ saved [1586544/1586544]

--2023-03-20 13:57:06--  https://raw.githubusercontent.com/usydnlp/Glocal_K/main/data/MovieLens_100K/movielens_100k_u1.test
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent

In [2]:
import time
from scipy.sparse import csc_matrix
import numpy as np
import h5py
import math as m

In [3]:
def load_data_100k(path='./', delimiter='\t'):

    train = np.loadtxt(path+'movielens_100k_u1.base', skiprows=0, delimiter=delimiter).astype('int32')
    test = np.loadtxt(path+'movielens_100k_u1.test', skiprows=0, delimiter=delimiter).astype('int32')
    total = np.concatenate((train, test), axis=0)

    n_u = np.unique(total[:,0]).size  # num of users
    n_m = np.unique(total[:,1]).size  # num of movies
    n_train = train.shape[0]  # num of training ratings
    n_test = test.shape[0]  # num of test ratings

    train_r = np.zeros((n_m, n_u), dtype='float32')
    test_r = np.zeros((n_m, n_u), dtype='float32')

    for i in range(n_train):
        train_r[train[i,1]-1, train[i,0]-1] = train[i,2]

    for i in range(n_test):
        test_r[test[i,1]-1, test[i,0]-1] = test[i,2]

    train_m = np.greater(train_r, 1e-12).astype('float32')  # masks indicating non-zero entries
    test_m = np.greater(test_r, 1e-12).astype('float32')

    print('data matrix loaded')
    print('num of users: {}'.format(n_u))
    print('num of movies: {}'.format(n_m))
    print('num of training ratings: {}'.format(n_train))
    print('num of test ratings: {}'.format(n_test))

    return n_m, n_u, train_r, train_m, test_r, test_m

In [4]:
# Insert the path of a data directory by yourself (e.g., '/content/.../data')
data_path = './'

In [5]:
n_m, n_u, train_r, train_m, test_r, test_m = load_data_100k(path=data_path, delimiter='\t')

data matrix loaded
num of users: 943
num of movies: 1682
num of training ratings: 80000
num of test ratings: 20000


In [6]:
test_m.shape

(1682, 943)

In [7]:
train_m.shape

(1682, 943)

## Some function for both algorithms

In [8]:
def calc_average_rating(data: np.ndarray, data_mask: np.ndarray, n_user: int) -> np.ndarray:
    result = np.empty(n_user, dtype='float32')
    for i in range(n_user):
        total_rating_score = data[:, i].sum()
        total_rating_action = data_mask[:, i].sum()
        result[i] = (total_rating_score / total_rating_action)
    return result

In [9]:
def rmse(predicted: np.ndarray, test:np.ndarray, test_m:np.ndarray, n_user, n_item) -> float:
  pred_flat = []
  test_flat = []
  for item in range(n_item):
    for user in range(n_user):
      if test_m[item][user] == 1:
        pred_flat.append(predicted[item][user])
        test_flat.append(test[item][user])
  pred_flat = np.asarray(pred_flat)
  test_flat = np.asarray(test_flat)
  return np.sqrt(np.mean((pred_flat-test_flat)**2))

## UBCF

In [10]:
def calc_user_sim(data: np.ndarray, data_mask: np.ndarray, average_rating: np.ndarray, n_item: int,
             user_u: int,
             user_v: int) -> float:
    if user_u == user_v:
        return -1
    common_item = []
    for i in range(n_item):
        if data_mask[i][user_u] and data_mask[i][user_v]:
            common_item.append(i)
    if common_item:
        numerator = 0.0
        denominator_left = 0.0
        denominator_right = 0.0
        for item in common_item:
            numerator += (data[item][user_u] - average_rating[user_u]) * (data[item][user_v] - average_rating[user_v])
            denominator_left += m.pow(data[item][user_u] - average_rating[user_u], 2)
            denominator_right += m.pow(data[item][user_v] - average_rating[user_v], 2)
        denominator = m.sqrt(denominator_left) * m.sqrt(denominator_right)
        if denominator == 0 :
            denominator = 0.001
        return numerator / denominator
    else:
        return - 1

In [11]:
def ubcf_predict_rating(data: np.ndarray, sims: np.ndarray, top_sims: np.ndarray, top_k: int, item: int, user: int, average_ratings: list) -> float:
    numerator = 0.0
    denominator = 0.0
    has_suitable_sims = False
    top_sim_users = top_sims[user]
    for k in range(top_k):
        sims_user_uv = sims[top_sim_users[k]][user]
        if k < len(top_sim_users) and sims_user_uv > 0:
            has_suitable_sims = True
            numerator += (data[item][top_sim_users[k]] - average_ratings[top_sim_users[k]])* sims_user_uv
            denominator += np.abs(sims_user_uv)
    if not has_suitable_sims:
      return 0
    predict = average_ratings[user]
    if denominator != 0:
        predict += numerator / denominator
    return predict

In [12]:
def ubcf(data: np.ndarray, data_mask: np.ndarray, n_user: int, n_item: int, top_k: int, test_mask: np.ndarray = None):
    big_tic = time.perf_counter()
    average_ratings = calc_average_rating(data, data_mask, n_user)

    #calculate sims
    print("calculating sims of all users to each other....")
    sims = np.zeros((n_user, n_user), dtype='float32')
    tic = time.perf_counter()
    for i in range(n_user - 1):
        for j in range(i + 1, n_user):
            sims[j][i] = sims[i][j] = calc_user_sim(data, data_mask, average_ratings, n_user, i, j)
    toc = time.perf_counter()
    duration = toc - tic
    print(f"calculated all sims in {duration:0.2f} seconds | {(duration / 60):0.2f} minutes")

    # sort top sims
    print(f"sorting users with top sim....")
    top_sims = np.empty((n_user, n_user - 1), dtype='int32')
    tic = time.perf_counter()
    for user in range(n_user):
        user_sims = sims[user]
        sorted_by_index = sorted(range(len(user_sims)), key=lambda t: user_sims[t], reverse=True)
        sorted_by_index.remove(user)
        top_sims[user] = sorted_by_index
    toc = time.perf_counter()
    duration = toc - tic
    print(f"sorted users with top sim in {duration:0.2f} seconds | {(duration / 60):0.2f} minutes")
    print(f"predicting missing rating values...")

    # predict
    tic = time.perf_counter()
    rating_predicted = np.zeros((n_item, n_user), dtype='float32')

    for user in range(n_user):
        for item in range(n_item):
            test_mask_based = test_mask is not None
            if test_mask_based:
              need_calc = test_mask[item][user] == 1
            else:
              need_calc = data_mask[item][user] == 0
            if need_calc:
                predict = ubcf_predict_rating(data, sims, top_sims, top_k, item, user, average_ratings)
                rating_predicted[item][user] = predict
                  

    toc = time.perf_counter()
    duration = toc - tic
    print(f"ratings value predicted in {duration:0.2f} seconds | {(duration / 60):0.2f} minutes")
    big_toc = time.perf_counter()
    big_duration = big_toc - big_tic
    print(f"all processes finished in {big_duration:0.2f} seconds | {(big_duration / 60):0.2f} minutes")
    return rating_predicted, top_sims

In [13]:
ubcf_predicted, ubcf_tops = ubcf(train_r, train_m, n_u, n_m, 2, test_m)

calculating sims of all users to each other....
calculated all sims in 151.97 seconds | 2.53 minutes
sorting users with top sim....
sorted users with top sim in 0.40 seconds | 0.01 minutes
predicting missing rating values...
ratings value predicted in 5.62 seconds | 0.09 minutes
all processes finished in 158.04 seconds | 2.63 minutes


In [14]:
ubcf_predicted

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [15]:
ubcf_predicted.shape

(1682, 943)

In [16]:
ubcf_tops

array([[  3,  38,  46, ..., 810, 855, 872],
       [  4,  21,  24, ..., 854, 913, 924],
       [  4,  19,  21, ..., 917, 926, 931],
       ...,
       [  9,  18,  21, ..., 917, 924, 928],
       [ 49, 149, 161, ..., 764, 854, 860],
       [  2,  73,  77, ..., 914, 919, 925]], dtype=int32)

## IBCF

In [17]:
def calc_item_sim(data: np.ndarray, data_mask: np.ndarray, average_rating: np.ndarray, n_user: int,
             item_i: int,
             item_j: int) -> float:
    if item_i == item_j:
        return -1
    common_user = []
    for i in range(n_user):
        if data_mask[item_i][i] and data_mask[item_j][i]:
            common_user.append(i)
    if common_user:
        numerator = 0.0
        denominator_left = 0.0
        denominator_right = 0.0
        for user in common_user:
            numerator += (data[item_i][user] - average_rating[user]) * (data[item_j][user] - average_rating[user])
            denominator_left += m.pow(data[item_i][user] - average_rating[user], 2)
            denominator_right += m.pow(data[item_j][user] - average_rating[user], 2)
        denominator = m.sqrt(denominator_left) * m.sqrt(denominator_right)
        if denominator == 0:
            denominator = 0.001
        return numerator / denominator
    else:
        return - 1


In [18]:
def ibcf_predict_rating(data: np.ndarray, sims: np.ndarray, top_sims: np.ndarray, top_k: int, item: int, user: int) -> float:
    numerator = 0.0
    denominator = 0.0
    has_suitable_sims = False
    top_sim_items = top_sims[item]
    for k in range(top_k):
        sims_item_ij = sims[item][top_sim_items[k]]
        if k < len(top_sim_items) and sims_item_ij > 0:
            has_suitable_sims = True
            numerator += data[top_sim_items[k]][user] * sims_item_ij
            denominator += np.abs(sims_item_ij)
    if not has_suitable_sims:
      return 0
    predict = 5
    if denominator != 0:
        predict = numerator / denominator
    return predict

In [19]:
def ibcf(data: np.ndarray, data_mask: np.ndarray,n_user: int, n_item: int, top_k,  test_mask: np.ndarray = None):
    big_tic = time.perf_counter()
    average_ratings = calc_average_rating(data, data_mask, n_user)

    #calculate sims
    print("calculating sims of all items to each other....")
    sims = np.zeros((n_item, n_item), dtype='float32')
    tic = time.perf_counter()
    for i in range(n_item - 1):
        for j in range(i + 1, n_item):
            sims[j][i] = sims[i][j] = calc_item_sim(data, data_mask, average_ratings, n_user, i, j)
    toc = time.perf_counter()
    duration = toc - tic
    print(f"calculated all sims in {duration:0.2f} seconds | {(duration / 60):0.2f} minutes")

    # sort top sims
    print(f"sorting items with top sim....")
    top_sims = np.empty((n_item, n_item - 1), dtype='int32')
    tic = time.perf_counter()
    for item in range(n_item):
        item_sims = sims[item]
        sorted_by_index = sorted(range(len(item_sims)), key=lambda t: item_sims[t], reverse=True)
        sorted_by_index.remove(item)
        top_sims[item] = sorted_by_index
    toc = time.perf_counter()
    duration = toc - tic
    print(f"sorted items with top sim in {duration:0.2f} seconds | {(duration / 60):0.2f} minutes")
    print(f"predicting missing rating values...")

    #predict 
    tic = time.perf_counter()
    rating_predicted = np.zeros((n_item, n_user), dtype='float32')
    for item in range(n_item):
        for user in range(n_user):
            test_mask_based = test_mask is not None
            if test_mask_based:
              need_calc = test_mask[item][user] == 1
            else:
              need_calc = data_mask[item][user] == 0
            if need_calc:
                predict = ibcf_predict_rating(data, sims, top_sims, top_k, item, user)
                rating_predicted[item][user] = predict
    toc = time.perf_counter()
    duration = toc - tic
    print(f"ratings value predicted in {duration:0.2f} seconds | {(duration / 60):0.2f} minutes")
    
    big_toc = time.perf_counter()
    big_duration = big_toc - big_tic
    print(f"all processes finished in {big_duration:0.2f} seconds | {(big_duration / 60):0.2f} minutes")
    return rating_predicted, top_sims

In [20]:
ibcf_predicted, ibcf_tops = ibcf(train_r, train_m, n_u, n_m, 2, test_m)

calculating sims of all items to each other....
calculated all sims in 431.65 seconds | 7.19 minutes
sorting items with top sim....
sorted items with top sim in 2.03 seconds | 0.03 minutes
predicting missing rating values...
ratings value predicted in 4.12 seconds | 0.07 minutes
all processes finished in 437.85 seconds | 7.30 minutes


In [21]:
ibcf_predicted

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [22]:
ibcf_predicted.shape

(1682, 943)

In [23]:
ibcf_tops

array([[ 340,  359,  438, ..., 1678, 1679, 1681],
       [ 112,  295,  307, ..., 1677, 1678, 1679],
       [  17,  118,  145, ..., 1678, 1679, 1680],
       ...,
       [ 258,  291,  298, ..., 1676, 1680, 1681],
       [   0,    1,    3, ..., 1678, 1679, 1681],
       [   1,    2,    4, ..., 1678, 1679, 1680]], dtype=int32)

## Evaluation metric

### 1/ RMSE

In [24]:
rmse(ubcf_predicted, test_r, test_m, n_u, n_m)

3.697425

In [25]:
rmse(ibcf_predicted, test_r, test_m, n_u, n_m)

3.66814

### 2/ NDCG

#### Version 1: Compute NDCG on merged dataset, included train, test and predicted dataset

In [26]:
def ndcg(i_rating, p_rating, k = None):
  upper = k
  if not upper or k > len(i_rating):
    upper = len(i_rating)
  idcg = 0.0
  dcg = 0.0
  for i in range(0, upper):
    idcg += i_rating[i] / m.log(i+2)
    dcg += p_rating[i] / m.log(i+2)
  if idcg == 0:
    return 0  
  return dcg / idcg

In [27]:
def ordered_items_by_rating(data: np.ndarray, n_user:int, n_item:int):
  ordered = np.empty((n_user, n_item), dtype='int32')
  for user in range(n_user):
    items_ratings = data[:, user]
    sorted_by_index = sorted(range(len(items_ratings)), key=lambda t: items_ratings[t], reverse=True)
    ordered[user] = sorted_by_index
  return ordered


In [28]:
def merge_data(data : np.ndarray, data_mask : np.ndarray, fill:np.ndarray, fill_mask :np.ndarray, n_user:int, n_item : int):
  merged = np.copy(data)
  for item in range(n_item):
    for user in range(n_user):
      if not data_mask[item][user] and fill_mask[item][user]:
        merged[item][user] = fill[item][user] 
  return merged


In [29]:
def calc_ndcgs(idea_items_order: np.ndarray, predicted_items_order: np.ndarray, data:np.ndarray,data_mask: np.ndarray ,n_user:int, n_item:int, k = None):
  ndcgs = np.zeros(n_user)
  for user in range(n_user):
    true_rating = []
    pred_rating = []
    for item in range(n_item):
      idea_item_index = idea_items_order[user][item]
      predicted_item_index = predicted_items_order[user][item]
      if data_mask[idea_item_index][user] == 0:
        continue
      true_rating.append(data[idea_item_index][user])
      pred_rating.append(data[predicted_item_index][user])
    ndcgs[user] = ndcg(true_rating, pred_rating, k)
  return ndcgs

merge full dataset

In [30]:
merged_data_full = merge_data(train_r, train_m, test_r, test_m, n_u, n_m)
merged_data_full_m = np.greater(merged_data_full, 1e-12)

In [31]:
merged_data_full_m.sum()

100000

In [32]:
idea_order = ordered_items_by_rating(merged_data_full, n_u, n_m)

In [33]:
idea_order

array([[   0,    5,    8, ..., 1679, 1680, 1681],
       [  49,   99,  126, ..., 1679, 1680, 1681],
       [ 319,  320,  327, ..., 1679, 1680, 1681],
       ...,
       [   0,  116,  123, ..., 1679, 1680, 1681],
       [  30,   49,   70, ..., 1679, 1680, 1681],
       [   1,   11,   41, ..., 1679, 1680, 1681]], dtype=int32)

NDCG on UBCF prediction

In [34]:
merged_data_pred_ubcf = merge_data(train_r, train_m, ubcf_predicted, test_m, n_u, n_m)
ubcf_pred_order = ordered_items_by_rating(merged_data_pred_ubcf, n_u, n_m)

In [35]:
ubcf_pred_order

array([[   0,    8,   12, ...,  264,  265,  266],
       [  49,   99,  126, ..., 1679, 1680, 1681],
       [ 319,  320,  339, ...,  349,  350,  353],
       ...,
       [   0,  116,  123, ..., 1679, 1680, 1681],
       [  30,   49,   70, ..., 1679, 1680, 1681],
       [   1,   11,   41, ..., 1679, 1680, 1681]], dtype=int32)

In [36]:
calculated_ndcgs = calc_ndcgs(idea_order, ubcf_pred_order, merged_data_full, merged_data_full_m, n_u, n_m, 100)

In [37]:
calculated_ndcgs.shape

(943,)

In [38]:
calculated_ndcgs[:10]

array([0.91862532, 0.99074315, 0.70053057, 0.99112671, 0.79136926,
       0.889439  , 0.96915564, 0.97348977, 0.73997092, 0.93457733])

NDCG on IBCF prediction

In [39]:
merged_data_pred_ibcf = merge_data(train_r, train_m, ibcf_predicted, test_m, n_u, n_m)
ibcf_pred_order = ordered_items_by_rating(merged_data_pred_ibcf, n_u, n_m)

In [40]:
ibcf_pred_order

array([[   0,    8,   12, ..., 1679, 1680, 1681],
       [  99,  126,  241, ..., 1679, 1680, 1681],
       [ 319,  320,  339, ..., 1679, 1680, 1681],
       ...,
       [   0,  116,  123, ..., 1679, 1680, 1681],
       [  30,   49,   70, ..., 1679, 1680, 1681],
       [   1,   11,   41, ..., 1679, 1680, 1681]], dtype=int32)

In [41]:
calculated_ndcgs_ibcf = calc_ndcgs(idea_order, ibcf_pred_order, merged_data_full, merged_data_full_m, n_u, n_m, 100)


In [42]:
calculated_ndcgs_ibcf.shape


(943,)

In [43]:
calculated_ndcgs_ibcf[:10]

array([0.92024455, 0.77879817, 0.69114239, 0.71332149, 0.78681996,
       0.88462275, 0.96915564, 0.65479985, 0.70496602, 0.9237595 ])

#### Version 2: Compute NDCG on just test and prediction dataset

In [44]:
def ordered_item_by_rating(rating: np.ndarray):
  item_ordered = []
  for i in range(len(rating.T)):
    list_rated_item = [j for j in range(len(rating.T[i])) if rating.T[i][j] != 0]
    if not list_rated_item: sorted_item = []
    else:
      list_rating = [rating.T[i][item] for item in list_rated_item]
      sorted_item = [item for _, item in sorted(zip(list_rating, list_rated_item),reverse=True)]
    item_ordered.append(np.array(sorted_item))
  return np.array(item_ordered)

In [45]:
def ndcg(true_idea_rating_ordered, true_rating_ordered_by_recommender, k = None):
  upper = len(true_rating_ordered_by_recommender)
  if k and k < upper: upper = k
  idcg = 0.0
  dcg = 0.0
  for i in range(0, upper):
    idcg += true_idea_rating_ordered[i] / m.log(i+2)
    dcg += true_rating_ordered_by_recommender[i] / m.log(i+2)
  if idcg == 0:
    return 0
  return dcg / idcg

In [46]:
def calc_ndcgs(real_rating: np.ndarray, idea_items_ordered: np.ndarray, pred_item_ordered: np.ndarray, n_users: int, k = None):
  ndcgs = []
  for i in range(n_users):
    true_idea_rating_ordered = [real_rating.T[i][item] for item in idea_items_ordered[i]]
    if true_idea_rating_ordered:
      true_rating_ordered_by_recommender = [real_rating.T[i][item] for item in pred_item_ordered[i]]
      ndcgs.append(ndcg(true_idea_rating_ordered,true_rating_ordered_by_recommender,k))
  return np.array(ndcgs)

NCDG on UBCF prediction

In [47]:
idea_item_ordered = ordered_item_by_rating(test_r)
pred_item_ordered = ordered_item_by_rating(ubcf_predicted)

<ipython-input-44-d298ff7e8b3d>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(item_ordered)


In [48]:
ndcgs = calc_ndcgs(test_r,idea_item_ordered, pred_item_ordered,n_u,5)
ndcgs[:10]

array([0.75519297, 0.7964244 , 0.61321538, 0.89877541, 0.59971992,
       0.61008746, 0.75515624, 0.78022776, 0.8474996 , 0.73216796])

NDCG on IBCF prediciton

In [49]:
idea_item_ordered = ordered_item_by_rating(test_r)
pred_item_ordered = ordered_item_by_rating(ibcf_predicted)

<ipython-input-44-d298ff7e8b3d>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(item_ordered)


In [50]:
ndcgs = calc_ndcgs(test_r,idea_item_ordered, pred_item_ordered,n_u,5)
ndcgs[:10]

array([0.61265743, 0.        , 0.        , 0.        , 0.        ,
       0.95307213, 0.87968593, 1.        , 0.        , 0.70614425])